In [ ]:
#title           :fc_py3.ipynb
#description     :This downloads future 5-day weather forecast, updating per 3 hours.
#author          :Yuanyuan Zhao
#date            :20181003
#version         :3
#notes           :
#python_version  :3.6.4
#==============================================================================

from datetime import datetime, date, time, timedelta
import time as t1
import requests
import bs4
import json
import yaml
import csv

# scrapes weather forecast data from openweathermap.org
def spider(path):
    
    global fcResult 
    fcResult = []
    locs = []
#     reads in locations
    with open(path, "r") as csvf:
        dataReader = csv.reader(csvf)
        for line in dataReader:
            locs.append(line)
            
    for loc in locs:
        url = "http://api.openweathermap.org/data/2.5/forecast?lat=%s&lon=%s&APPID=ea4985020f724407dea8833c9dfee64c"%(loc[0], loc[1])
        response = requests.get(url)
        json_data = yaml.load(json.dumps(response.json()))
#         stores data in the following format: 
#         [dt, temp, pressure, humidity, temp_min, temp_max, wind_speed, clouds, weather_id, weather, description]
        for x in json_data['list']:
            weather_data = ((item["id"], item["main"], item["description"]) for item in x["weather"])
            fcResult.append([datetime.utcfromtimestamp(int(x["dt"])).strftime('%Y-%m-%d %H:%M:%S'),
                        x["main"]["temp"],
                        x["main"]["pressure"],
                        x["main"]["humidity"],
                        x["main"]["temp_min"],
                        x["main"]["temp_max"],
                        x["wind"]["speed"],
                        x["clouds"]["all"],
                        list(weather_data)
                        ])
    return fcResult

while(True):
#     forecast weather data for each path
    fc_AUR02_CAR01 = spider('AUR02-CAR01.csv')
    fc_CHI02_AUR02 = spider('CHI02-AUR02.csv')
    fc_FRA01_SLO02 = spider('FRA01-SLO02.csv')
    fc_SEC10_TOR01 = spider('SEC10-TOR01.csv')
#     update every 3 hour
    t1.sleep(60*60*3)
